In [1]:
!pip install streamlit
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 931.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 4s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [2]:
!pip install streamlit
!pip install pyngrok

In [3]:
!pip install openpyxl

In [12]:
%%writefile app.py

import pandas as pd
import streamlit as st

st.set_page_config(page_title="Stage Shift Dashboard", layout="wide")
st.title("📊 Stage Shift Comparison Dashboard")

st.markdown("Upload the **previous** and **current** Excel sheets to analyze new registrations and status transitions.")

# Upload previous and current sheets
prev_file = st.file_uploader("Upload Previous Sheet (e.g. April 1)", type=["xlsx"], key="prev")
curr_file = st.file_uploader("Upload Current Sheet (e.g. April 2)", type=["xlsx"], key="curr")

# Function to determine the user's stage based on statuses
def determine_stage(row):
    if row.get("Payment_Status") == "Completed":
        return "Payment"
    elif row.get("Upload_Status") == "Completed":
        return "Upload"
    elif row.get("Academic_Status") == "Completed":
        return "Academic"
    elif row.get("Personal_Status") == "Completed":
        return "Personal"
    else:
        return "Registered"

if prev_file and curr_file:
    prev_df = pd.read_excel(prev_file)
    curr_df = pd.read_excel(curr_file)

    if "Email_ID" not in prev_df.columns or "Email_ID" not in curr_df.columns:
        st.error("⚠️ 'Email_ID' column must be present in both sheets.")
    else:
        # Calculate stage for previous and current data
        prev_df["Stage"] = prev_df.apply(determine_stage, axis=1)
        curr_df["Stage"] = curr_df.apply(determine_stage, axis=1)

        # Total registered users in previous sheet
        total_prev = prev_df["Email_ID"].nunique()

        # New users in current sheet (not present in previous)
        new_users_df = curr_df[~curr_df["Email_ID"].isin(prev_df["Email_ID"])]
        total_new = new_users_df["Email_ID"].nunique()

        # Summary table for registration data
        summary_df = pd.DataFrame({
            "Metric": ["Total in Previous Sheet", "New Users in Current Sheet"],
            "Count": [total_prev, total_new]
        })

        st.subheader("📌 Registration Summary")
        st.dataframe(summary_df, use_container_width=True)

        # Merge previous and current data on Email_ID to determine stage transitions
        merged = pd.merge(prev_df[["Email_ID", "Stage"]],
                          curr_df[["Email_ID", "Stage"]],
                          on="Email_ID", suffixes=("_prev", "_curr"))

        # Filter to only those with changes in stage
        stage_shift_df = merged[merged["Stage_prev"] != merged["Stage_curr"]]
        shift_counts = stage_shift_df.groupby(["Stage_prev", "Stage_curr"]).size().reset_index(name="Count")

        st.subheader("🔁 Stage Transitions")
        st.dataframe(shift_counts, use_container_width=True)

        # Download stage shift report as CSV
        csv = shift_counts.to_csv(index=False).encode("utf-8")
        st.download_button("⬇️ Download Stage Shift Report (CSV)", csv, file_name="stage_shift_report.csv")

else:
    st.info("👈 Upload both sheets to begin analysis.")


Overwriting app.py
